In [1]:
import random
import matplotlib.pyplot as plt


In [2]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA


In [3]:
linguistic_variable = "linguistic variable"
Paramerters_S = "S"
Paramerters_M = "M"
y = "y"
type_Fuzzy_Set = "fuzzy set"
rule = "rule"   
structure = 'structure'
being = 'being'
not_list = 'not_list'
term_choice = 'term_choice'
elements = 'elements'
fitness = 'fitness'
gens = 'gens'
gen_num = 5

Max_S = 70   # 2 * 35   # calcualte the database
Min_S = 0
Max_M = 7    # 2*5
Min_M = -11  # 2*-5
element_num = 10
max_num_of_features = 5
linguistic = ["very low", "low", "medium", "high", "very high"]
num_of_chromosoms = 10

crossover_rate = 0.8
mutation_rate = 0.2

## Extraxt data


In [4]:
# read and pre process the data set
def process_data(sms_data_str):
    """
    convert `sms_data_str` into a pandas dataframe
    """
    data_arr = []

    data_records = sms_data_str.split('\n')[:-1]
    for data in data_records:
        label = None
        sample = None
        data_type = data[:3]
        if data_type == 'ham':
            label = 'legitimate'
            sample = data[4:]
        elif data_type == 'spa':
            label = 'spam'
            sample = data[5:]
        else:
            label = 'N/A'

        data_arr.append([label, sample])

    data_arr = np.array(data_arr)
    data_label = data_arr[:, 0]
    data_records = data_arr[:, 1]

    return data_records, data_label
# extract feature from SMS


def tfidf_vectorizer(records):
    vectorizer = TfidfVectorizer(
        lowercase=True,
        token_pattern=r'\b[A-Za-z]+\b',
        norm=None
    )

    records_transformed = vectorizer.fit_transform(records)

    return records_transformed.toarray(), vectorizer.get_feature_names_out()


# decrece the dimension
def feature_extraction(X, n_components=5):
    reduction_pca = PCA(
        n_components=n_components,
        whiten=False
    )
    data_reduced = reduction_pca.fit_transform(X)
    return data_reduced

# select the feature


def feature_selection(df_records, labels, n_components=5):
    feature_selection_model = SelectKBest(mutual_info_classif, k=n_components)
    # make a selection over the best features
    selected_record_features = feature_selection_model.fit_transform(
        df_records, labels)

    return selected_record_features, feature_selection_model.get_feature_names_out()


In [5]:
# read data
sms_data_str = None
with open('SMSSpamCollection') as file:
    sms_data_str = file.read()


In [6]:
# extract feature
records, labels = process_data(sms_data_str)
records_vectorized, feature_names = tfidf_vectorizer(records)

# one hot encoding labels (convert label to 0 or 1)
labels = np.array([0 if y == 'legitimate' else 1 for y in labels])

# reducing dimension
records_dim_reduced = feature_extraction(records_vectorized)


In [7]:
# feature extraxtion
print(records_dim_reduced[0])
print(records_dim_reduced[1])
print(records_dim_reduced[1][0])
print(records_dim_reduced.shape)


[-1.8563898   0.28359711 -1.18528835  0.82070463  0.73044192]
[-2.78398934  0.52118069 -1.74248647  0.50102967 -0.7376764 ]
-2.7839893399905087
(5574, 5)


In [8]:
# read data
# open the file for reading
labels = []
with open('SMSSpamCollection', 'r') as f:

    for line in f:
        # split the line into words
        words = line.split()
        # select the first word
        first_word = words[0]
        if words[0] == 'ham':
            labels.append(0)
        else:
            labels.append(1)

print(labels)
# find number of zero in labels
print(labels.count(0))
# find number of one in labels
print(labels.count(1))

# find the index of zero in labels
zero_index = [i for i, x in enumerate(labels) if x == 1]
print(zero_index)
# find the index of one in labels
one_index = [i for i, x in enumerate(labels) if x == 0]


[0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
# # This function creates a list of chromosomes.
# def create_chromosomes2(num_chromosomes):
#     # Initialize an empty list to store the chromosomes.
#     chromosomes = []

#     # For each chromosome:
#     for i in range(num_chromosomes):
#         # Initialize an empty dictionary to store this chromosome's genetic material.
#         chro = dict()
#         chro[structure] = []
#         # For each genetic element (gene) in the chromosome:
#         for _ in range(gen_num):
#             # Initialize an empty dictionary to store this gene's features (linguistic variable, parameter S, parameter M, and type of fuzzy set).
#             gen = dict()
#             # Generate a random number between 3 and 5 to determine the number of linguistic variables in this feature.
#             num_feature = random.randint(3, 5)
#             # For each linguistic variable in this feature, randomly choose one from the set of possible linguistic variables and add it to the list of linguistic variables in this feature.
#             gen[linguistic_variable] = [random.choice(
#                 linguistic) for i in range(num_feature)]
#             # For each parameter S in this feature, randomly generate a floating-point number between Min_S and Max_S and add it to the list of S parameters in this feature.
#             gen[Paramerters_S] = [random.uniform(
#                 Min_S, Max_S) for i in range(num_feature)]
#             # For each parameter M in this feature, randomly generate a floating-point number between Min_M and Max_M and add it to the list of M parameters in this feature.
#             gen[Paramerters_M] = [random.uniform(
#                 Min_M, Max_M) for i in range(num_feature)]
#             # For each type of fuzzy set in this feature, randomly generate an integer between 1 and 4 and add it to the list of fuzzy set types in this feature.
#             gen[type_Fuzzy_Set] = [random.randint(
#                 1, 4) for i in range(num_feature)]
#             # Add the current gene (with its features) to this chromosome.
#             chro[structure].append(gen)

#         # Initialize an empty dictionary to store the rule for this chromosome's genetic material.
#         gen_rule = dict()
#         # For each genetic element (gene) in the chromosome:
#         gen_rule[being] = [random.choice([0, 1]) for i in range(
#             gen_num)]     # 1 ==> be   0 ==> not be
#         gen_rule[not_list] = [random.choice([0, 1]) for i in range(
#             gen_num)]  # 1 ==> NOT   0 ==> self
#         gen_rule[term_choice] = [random.choice(
#             linguistic) for i in range(gen_num)]   # which term?
#         gen_rule[y] = random.choice([0, 1])   # spam or ham
#         # Add the rule to this chromosome.
#         chro[rule] = gen_rule
#         print("HI")
#         chro[fitness] = 0
#         # Add this chromosome (with its genetic material and rule) to the list of chromosomes.
#         chromosomes.append(chro)

#     # Return the list of chromosomes.
#     return chromosomes


In [10]:
# This function creates a list of chromosomes.
def create_chromosomes(num_chromosomes):
    # Initialize an empty list to store the chromosomes.
    chromosomes = []
    
    # For each chromosome:
    for i in range(num_chromosomes):
        # Initialize an empty dictionary to store this chromosome's genetic material.
        chro = dict()
        chro[structure] = []
        # For each genetic element (gene) in the chromosome:
        for _ in range(gen_num):
            # Initialize an empty dictionary to store this gene's features (linguistic variable, parameter S, parameter M, and type of fuzzy set).
            gen = dict()
            # Generate a random number between 3 and 5 to determine the number of linguistic variables in this feature.
            num_feature = random.randint(3, 5)
            # For each linguistic variable in this feature, randomly choose one from the set of possible linguistic variables and add it to the list of linguistic variables in this feature.
            gen[linguistic_variable] = [random.choice(linguistic) for i in range(num_feature)]
            # For each parameter S in this feature, randomly generate a floating-point number between Min_S and Max_S and add it to the list of S parameters in this feature.
            gen[Paramerters_S] = [random.uniform(Min_S, Max_S) for i in range(num_feature)]
            # For each parameter M in this feature, randomly generate a floating-point number between Min_M and Max_M and add it to the list of M parameters in this feature.
            gen[Paramerters_M] = [random.uniform(Min_M, Max_M) for i in range(num_feature)]
            # For each type of fuzzy set in this feature, randomly generate an integer between 1 and 4 and add it to the list of fuzzy set types in this feature.
            gen[type_Fuzzy_Set] = [random.randint(1, 4) for i in range(num_feature)]
            # Add the current gene (with its features) to this chromosome.
            chro[structure].append(gen)
        
        # Initialize an empty dictionary to store the rule for this chromosome's genetic material.
        gen_rule = dict()
        # For each genetic element (gene) in the chromosome:
        gen_rule[being] = [random.choice([0, 1]) for i in range(gen_num)]     # 1 ==> be   0 ==> not be
        gen_rule[not_list] = [random.choice([0, 1]) for i in range(gen_num)]  # 1 ==> NOT   0 ==> self
        gen_rule[term_choice] = [random.choice(chro[structure][i][linguistic_variable]) for i in range(gen_num)]   # which term?
        gen_rule[y] = random.choice([0, 1])   # spam or ham
        # Add the rule to this chromosome.
        chro[rule] = gen_rule  
        
        chro[fitness] = 0
        # Add this chromosome (with its genetic material and rule) to the list of chromosomes.
        chromosomes.append(chro)
        
    # Return the list of chromosomes.
    return chromosomes

In [36]:

chro = create_chromosomes(2)
print(len(chro[0]))
# print(chro[0][structure][0][linguistic_variable])
# print(chro[1][structure][0])

parent1 = chro[0]
parent2 = chro[1]
print(parent1)
# print(parent2)
print(parent1[rule][term_choice])
print(parent1[rule][y])
print(parent1[structure][0][type_Fuzzy_Set])


3
{'structure': [{'linguistic variable': ['medium', 'medium', 'very low', 'very low', 'high'], 'S': [34.12704549955999, 39.49124169269421, 18.247339867987456, 35.36465063659856, 14.152526334508815], 'M': [5.984462930614097, 6.906142370636491, -0.16514061322526885, 4.166947318810838, -5.65015690800468], 'fuzzy set': [4, 4, 4, 2, 2]}, {'linguistic variable': ['medium', 'very high', 'low', 'very high', 'very low'], 'S': [15.514211770496406, 13.982677077093808, 20.613222770687678, 19.829226847236512, 38.1948075978943], 'M': [-10.997881496903934, -0.17637782206093178, 4.573738594129498, 0.29829960701084346, 6.341349480555575], 'fuzzy set': [1, 4, 1, 4, 1]}, {'linguistic variable': ['very high', 'very low', 'very high'], 'S': [42.55567500335085, 5.3563459308942285, 1.8736879505417614], 'M': [-7.075754652570278, 1.7762291229412472, 3.8426367525454435], 'fuzzy set': [3, 4, 1]}, {'linguistic variable': ['high', 'very low', 'high', 'medium', 'very low'], 'S': [39.50054523975096, 60.0654581299661

In [16]:
# This function takes in two parents (sequences) and creates two offsprings by performing a continuous crossover.
def crossover_continuous(parent1, parent2):    
    # Set the alpha value for the crossover.
    alpha = 0.25
    offspring1 = []
    offspring2 = []
    # If the first sequence is longer than the second one:
    if len(parent1) > len(parent2):
        # Generate a list of random indexes for the shorter sequence.
        indexes = random.sample(range(len(parent1)), len(parent2))
        # For each index in the shorter parent: if the corresponding element in the longer parent is greater, perform a crossover calculation using alpha and add the results to offspring1 and offspring2. Otherwise, perform the calculation using the reverse order and add the results to offspring1 and offspring2.
        for i in range(len(parent2)):
            if parent2[i] > parent1[indexes[i]]:
                distance = parent2[i] - parent1[indexes[i]]
                offspring1.append(parent1[indexes[i]] + alpha * distance)
                offspring2.append(parent2[i] - alpha * distance)
            else:
                distance = parent1[indexes[i]] - parent2[i]
                offspring1.append(parent2[i] + alpha * distance)
                offspring2.append(parent1[indexes[i]] - alpha * distance)
                
        # Add the remaining elements of the longer parent to the first offspring.     
        temp = [item for item in [_ for _ in range(len(parent2))] if item not in indexes]
        for i in range(len(temp)):
            offspring1.append(parent1[temp[i]])
        
    # If the second sequence is longer than or the same length as the first one:
    else:
        # Generate a list of random indexes for the shorter sequence.
        indexes = random.sample(range(len(parent2)), len(parent1))
        # For each index in the shorter parent: if the corresponding element in the longer parent is greater, perform a crossover calculation using alpha and add the results to offspring1 and offspring2. Otherwise, perform the calculation using the reverse order and add the results to offspring1 and offspring2.
        for i in range(len(parent1)):
            if parent1[i] > parent2[indexes[i]]:
                distance = parent1[i] - parent2[indexes[i]]
                offspring1.append(parent2[indexes[i]] + alpha * distance)
                offspring2.append(parent1[i] - alpha * distance)
            else:
                distance = parent2[indexes[i]] - parent1[i]
                offspring1.append(parent1[i] + alpha * distance)
                offspring2.append(parent2[indexes[i]] - alpha * distance)
                
        # Add the remaining elements of the longer parent to the second offspring. 
        temp = [item for item in [_ for _ in range(len(parent2))] if item not in indexes]
        for i in range(len(temp)):
            offspring2.append(parent2[temp[i]])
    
    # Return the two new offspring.
    return offspring1, offspring2

In [17]:
# This function takes in two parents (sequences) and creates two offsprings by performing a discrete crossover.
def crossover_discrete(parent1, parent2):   
    offspring1 = []
    offspring2 = []
    # If the first sequence is longer than the second one:
    if len(parent1) > len(parent2):
        # Generate a list of random indexes for the shorter sequence.
        indexes = random.sample(range(len(parent1)), len(parent2))
        # For each index in parent1: if it's in indexes, add the corresponding element to offspring2. Otherwise, add it to offspring1. 
        for i in range(len(parent1)):
            if i in indexes:
                offspring2.append(parent1[i])
            else:
                offspring1.append(parent1[i])
        # Add all of parent2 to offspring1.
        offspring1.extend(parent2)     
    # If the second sequence is longer than or the same length as the first one:
    else:
        # Generate a list of random indexes for the shorter sequence.
        indexes = random.sample(range(len(parent2)), len(parent1))
        # For each index in parent2: if it's in indexes, add the corresponding element to offspring1. Otherwise, add it to offspring2. 
        for i in range(len(parent2)):
            if i in indexes:
                offspring1.append(parent2[i])
            else:
                offspring2.append(parent2[i])
        # Add all of parent1 to offspring2.
        offspring2.extend(parent1)
    
    # Return the two new offspring.
    return offspring1, offspring2 


In [18]:
def crossover_term_choice(parent1, parent2):
    child1, child2 = crossover_discrete(parent1[rule][term_choice], parent2[rule][term_choice])
    for i in range(len(child1)):
        if child1[i] not in parent1[structure][i][linguistic_variable]:
            child1[i] = random.choice(parent1[structure][i][linguistic_variable])
    
    for i in range(len(child2)):
        if child2[i] not in parent2[structure][i][linguistic_variable]:
            child2[i] = random.choice(parent2[structure][i][linguistic_variable])
    
    return child1, child2

In [19]:
def crossover(parent1, parent2):
    structure1 = [dict() for i in range(max_num_of_features)]
    structure2 = [dict() for i in range(max_num_of_features)]
    
    for i in range(max_num_of_features):
        structure1[i][linguistic_variable], structure2[i][linguistic_variable] = crossover_discrete(parent1[structure][i][linguistic_variable], parent2[structure][i][linguistic_variable])
        structure1[i][type_Fuzzy_Set], structure2[i][type_Fuzzy_Set] = crossover_discrete(parent1[structure][i][type_Fuzzy_Set], parent2[structure][i][type_Fuzzy_Set])
        structure1[i][Paramerters_M], structure2[i][Paramerters_M] = crossover_continuous(parent1[structure][i][Paramerters_M], parent2[structure][i][Paramerters_M])
        structure1[i][Paramerters_S], structure2[i][Paramerters_S] = crossover_continuous(parent1[structure][i][Paramerters_S], parent2[structure][i][Paramerters_S])
    
    
    rule1 = dict()
    rule2 = dict()
    
    rule1[being], rule2[being] = crossover_discrete(parent1[rule][being], parent2[rule][being])
    rule1[not_list], rule2[not_list] = crossover_discrete(parent1[rule][not_list], parent2[rule][not_list])
    rule1[term_choice], rule2[term_choice] = crossover_term_choice(parent1, parent2)
    
    rule1[y], rule2[y] = parent2[rule][y], parent1[rule][y]
    
    offspring1 = dict()
    offspring1[structure] = structure1
    offspring1[rule] = rule1
    
    offspring2 = dict()
    offspring2[structure] = structure2
    offspring2[rule] = rule2
    
    return offspring1, offspring2

In [20]:
def mutation_linguistic(parent):
    for i in range(len(parent)):
        if random.uniform(0, 1) < mutation_rate:
            parent[i] = random.choice(linguistic)   
        else:
            pass
    return parent

In [21]:
def mutation_S(parent):
    for i in range(len(parent)):
        if random.uniform(0, 1) < mutation_rate:
            parent[i] = random.uniform(Max_S, Min_S)
        else:
            pass
    return parent

In [22]:
def mutation_M(parent):
    for i in range(len(parent)):
        if random.uniform(0, 1) < mutation_rate:
            parent[i] = random.uniform(Max_M, Min_M)
        else:
            pass
    return parent

In [23]:
def mutation_fuzzy(parent):
    for i in range(len(parent)):
        if random.uniform(0, 1) < mutation_rate:
            parent[i] = random.randint(0, 4) 
        else:
            pass
    return parent

In [24]:
def mutation_being_notList(parent):
    for i in range(len(parent[being])):
        if random.uniform(0, 1) < mutation_rate:
            parent[being][i] = parent[being][i] * -1 + 1
        else:
            pass
    
    for i in range(len(parent[not_list])):
        if random.uniform(0, 1) < mutation_rate:
            parent[not_list][i] = parent[not_list][i] * -1 + 1
        else:
            pass
        
    return parent

In [25]:
def mutation_term_choice(parent):
    for i in range(len(parent[rule][term_choice])):
        if random.uniform(0, 1) < mutation_rate:
            parent[rule][term_choice][i] = random.choice(parent[structure][i][linguistic_variable])
        else:
            pass
    return parent

In [26]:
def mutation(parent):
    for i in range(gen_num):
        parent[structure][i][linguistic_variable] = mutation_linguistic(parent[structure][i][linguistic_variable])
        parent[structure][i][type_Fuzzy_Set] = mutation_fuzzy(parent[structure][i][type_Fuzzy_Set])
        parent[structure][i][Paramerters_M] = mutation_M(parent[structure][i][Paramerters_M])
        parent[structure][i][Paramerters_S] = mutation_S(parent[structure][i][Paramerters_S])
    
    parent[rule][not_list] = mutation_being_notList(parent[rule])
    parent[rule][term_choice] = mutation_term_choice(parent)
    
    if random.uniform(0, 1) < mutation_rate:
        parent[rule][y] = parent[rule][y] * -1 + 1

    
    return parent

In [27]:
len(parent2[structure])


5

In [29]:
print("parent1 : ",parent1)
print("parent2 : ",parent2)
child1, child2 = crossover(parent1, parent2)
print("child1 cross : ",child1)
print("child2 cross : ",child2)
print(child1[structure][0][Paramerters_S])
print(child2[structure][0][Paramerters_S])
child1 = mutation(child1)
child2 = mutation(child2)
print("mutation")
print(child1[structure][0][Paramerters_S])
print(child2[structure][0][Paramerters_S])

parent1 :  {'structure': [{'linguistic variable': ['low', 'very high', 'high', 'very low'], 'S': [23.30076138740461, 10.45661655505471, 0.9622754716055371, 69.00919816606915], 'M': [-3.5437546884863647, -2.603793450865492, -4.158568461920029, 4.121059770437514], 'fuzzy set': [3, 2, 2, 2]}, {'linguistic variable': ['medium', 'low', 'very high'], 'S': [9.172740301317823, 4.1264542692924175, 25.26629450341109], 'M': [5.377055204097669, -8.908256319966075, 4.187818142958424], 'fuzzy set': [3, 2, 1]}, {'linguistic variable': ['high', 'medium', 'very high'], 'S': [24.27471628498425, 22.03703837309094, 50.74302679708914], 'M': [-8.143950665815629, -10.127193589361484, 0.6349144806541229], 'fuzzy set': [2, 2, 2]}, {'linguistic variable': ['medium', 'very low', 'very low'], 'S': [23.69735826332362, 16.951013505502882, 16.385406623592708], 'M': [2.425158562212193, -1.869139903294304, -0.936334985944427], 'fuzzy set': [2, 1, 1]}, {'linguistic variable': ['high', 'very low', 'very high', 'low', 'l

In [30]:

def isosceles_triangular_fuzzy(x, m, s):
    return np.max(np.min((x - m) / s, (m - x) / s), 0)


def trapezoids_of_Malzawie_fuzzy(x, m, s):
    return np.max(np.min((x - m) / s, 1), 0)


def gaussian_fuzzy(x, m, s):
    return np.exp(-0.5 * ((x - m) / s) ** 2)


def sigmoid_fuzzy(x, m, s):
    return 1 / (1 + np.exp((x - m) / s))
# def Fc(R,numberOfrule):


# def Fneg(Ruls,numberOfrule):
#     for i in range(numberOfrule):
#         pass

def gR(xP, muA):
    result = 1
    for i in range(len(xP)):
        result *= muA[i] * xP[i]
    return result


def gC(LOFL_xP, LOFL_muA, numberOfrule=5):  # nothing
    result = 0
    for i in range(len(LOFL_xP)):
        result += gR(LOFL_xP[i], LOFL_muA[i])
    return result


def Fc(LOFL_xP, LOFL_muA, numberOfrule):  #
    result = 0
    for i in range(len(LOFL_xP)):
        result += gR(LOFL_xP[i], LOFL_muA[i])
    return result


def Fneg(LOFL_xP, LOFL_muA, number_another_rule):   # r = number_another_rule

    return Fc(LOFL_xP, LOFL_muA, number_another_rule)


def Cf(LOFL_xP, LOFL_muA, numberOfrule):  # also we are detect gole class is zero or one
    fneg_Rj = Fneg(LOFL_xP, LOFL_xP, abs(5574-numberOfrule))
    fc_Rj = Fc(LOFL_xP, LOFL_muA, numberOfrule)
    return (fc_Rj - fneg_Rj) / (fc_Rj + fneg_Rj)


In [31]:
def find_Mu(xP, parent):  # parent is one chromosome
    Mu = []
    Mu_temp = []
    M = []
    S = []
    fuzzy_function = []   # [4, 3]
    indices_being = [j for j, x in enumerate(parent[rule][being]) if x == 0]
    for i in range(len(parent[structure])):
        if i in indices_being:
            pass
        else:
            # indices  list of index where have multi equal value  E.g == > [0, 1]
            indices_terms = [i for i, x in enumerate(parent[structure][i][linguistic_variable]) if x == parent[rule][term_choice][i]]
            indices_not_terms = [j for j, x in enumerate(parent[rule][not_list]) if x == 1]
            for j in range(indices_terms):  # find the fuzzy set  E.g [4, 3]
                fuzzy_function.append(parent[structure][i][type_Fuzzy_Set][indices_terms[j]])
                M.append(parent[structure][i][Paramerters_M][indices_terms[j]])
                S.append(parent[structure][i][Paramerters_S][indices_terms[j]])
            for j in range(len(fuzzy_function)): # select the function fuzzy
                if fuzzy_function[j] == 1:
                    Mu_temp.append(isosceles_triangular_fuzzy(xP[i], M[i], S[i]))
                elif fuzzy_function[j] == 2:
                    Mu_temp.append(trapezoids_of_Malzawie_fuzzy(xP[i], M[i], S[i]))
                elif fuzzy_function[j] == 3:
                    Mu_temp.append(gaussian_fuzzy(xP[i], M[i], S[i]))
                elif fuzzy_function[j] == 4:
                    Mu_temp.append(sigmoid_fuzzy(xP[i], M[i], S[i]))
            if i in indices_not_terms:
                Mu_temp = 1-Mu_temp
            Mu.append(max(Mu_temp))
            Mu_temp.clear()
            fuzzy_function.clear()
            S.clear()
            M.clear()

    return Mu


In [ ]:
def not_term(Mu, parent):
    indices_not = [j for j, x in enumerate(parent[rule][not_list]) if x == 1]
    for i in range(Mu):
        if i in indices_not:
            Mu[i] = 1 - Mu[i]
    
    return Mu

In [ ]:
def feed_data(parent):
    List_Of_List_xP = records_dim_reduced
    # List_Of_List_muA =
    # term_choice_select =
    # fuzzy_select =
    Mus = []
    for j in range(len(list_Of_List_xP)):
        Mu_temp = find_Mu(List_Of_List_xP[j], parent)
        # Mu = not_term(Mu_temp, parent)
        gr_xP = gR(List_Of_List_xP[j], Mu_xP)
        if parent[rule][y] ==  1:
            # index one label in data set == > variable one_index 
            # Fc = Fc(List_Of_List_xP, Mu, numberOfrule)
        else: 
            # index zero label in data set == > variable zero_index 
            # Fneg = Fneg(List_Of_List_xP, Mu, numberOfrule)
        
            
            
        
        # for i in range(num_gen):
        #     Mus.append(find_Mu(xP, parent))


In [ ]:
def evalate_fitness(population):
    for i in range(len(population)):
        population[i][fitness] = fitness_function(population[i])
